In [1]:
teamlink_nms = "https://www.primeleague.gg/leagues/teams/185633-nms-crime"
teamlink_sfd = 'https://www.primeleague.gg/leagues/teams/105731-sfd-gaming-snek-pack'
match_url = 'https://www.primeleague.gg/leagues/matches/930121-we-dont-know-vs-nms-crime' 